In [15]:
pip install pandas transformers


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/ytb_Its_A_Wonderful_Life.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(48, 2)


In [18]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

[{'label': '1', 'score': 0.9001131653785706},
 {'label': '2', 'score': 0.6775303483009338},
 {'label': '2', 'score': 0.5353849530220032},
 {'label': '2', 'score': 0.8662224411964417},
 {'label': '2', 'score': 0.7317637205123901},
 {'label': '2', 'score': 0.8659677505493164},
 {'label': '2', 'score': 0.6593604683876038},
 {'label': '2', 'score': 0.7765306234359741},
 {'label': '2', 'score': 0.7749651670455933},
 {'label': '2', 'score': 0.8657562136650085},
 {'label': '2', 'score': 0.7293841242790222},
 {'label': '0', 'score': 0.6381946802139282},
 {'label': '2', 'score': 0.7191898822784424},
 {'label': '2', 'score': 0.9315723776817322},
 {'label': '1', 'score': 0.4014383852481842},
 {'label': '2', 'score': 0.7535015940666199},
 {'label': '2', 'score': 0.6821113228797913},
 {'label': '2', 'score': 0.8971749544143677},
 {'label': '2', 'score': 0.9014561176300049},
 {'label': '2', 'score': 0.8415181636810303},
 {'label': '1', 'score': 0.36261075735092163},
 {'label': '2', 'score': 0.839360

In [19]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,This movie is not realistic enough. It should ...,1,0.900113
1,I love this movie so much but I like to think ...,2,0.677530
2,"<a href=""https://www.youtube.com/watch?v=SEgUp...",2,0.535385
3,“Shall I pray for him?” That was so sweet of h...,2,0.866222
4,Yeah this is so preferable to a peaceful life ...,2,0.731764
5,George Bailey It&#39;s me when i accused the w...,2,0.865968
6,Chuckie finster as George bailey.,2,0.659360
7,Who names their kid &quot;Zuzu&quot;?,2,0.776531
8,"As a man, if you ever wanna feel beyond guilty...",2,0.774965
9,"@<a href=""https://www.youtube.com/watch?v=SEgU...",2,0.865756


In [20]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         1            joy
1         2           love
2         2           love
3         2           love
4         2           love
5         2           love
6         2           love
7         2           love
8         2           love
9         2           love
10        2           love
11        0        sadness
12        2           love
13        2           love
14        1            joy
15        2           love
16        2           love
17        2           love
18        2           love
19        2           love
20        1            joy
21        2           love
22        2           love
23        2           love
24        2           love
25        1            joy
26        2           love
27        2           love
28        2           love
29        2           love
30        2           love
31        2           love
32        1            joy
33        2           love
34        1            joy
35        1            joy
3

<ipython-input-20-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [21]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_Its_A_Wonderful_Life_with_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,This movie is not realistic enough. It should ...,1,0.900113,joy
1,I love this movie so much but I like to think ...,2,0.677530,love
2,"<a href=""https://www.youtube.com/watch?v=SEgUp...",2,0.535385,love
3,“Shall I pray for him?” That was so sweet of h...,2,0.866222,love
4,Yeah this is so preferable to a peaceful life ...,2,0.731764,love
5,George Bailey It&#39;s me when i accused the w...,2,0.865968,love
6,Chuckie finster as George bailey.,2,0.659360,love
7,Who names their kid &quot;Zuzu&quot;?,2,0.776531,love
8,"As a man, if you ever wanna feel beyond guilty...",2,0.774965,love
9,"@<a href=""https://www.youtube.com/watch?v=SEgU...",2,0.865756,love
